In [1]:
import pandas as pd
import numpy as np
import nltk
import re
 
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk import word_tokenize

In [ ]:
#add precleaned old csv data file 
data  = pd.read_csv('/Users/urmi/Documents/NLP/Assignment_1/pmc-data-all-new.csv', encoding = "ISO-8859-1",converters={'Stemmed_abstract': eval})


In [ ]:
data = data.dropna()

In [ ]:
#data.reset_index(drop=True, inplace=True)
data.info()

In [ ]:
data = data.drop(['Unnamed: 0'], axis = 1) 
#data = data.drop(['Unnamed: 0.1'], axis = 1) 


In [ ]:
#data['Stop word Tokenized abstract'][0]

In [ ]:
#one = data[data['Article_Id']==3446065]
#print(one)
#data.iloc[1785355].Article_Id

In [ ]:
#type(data['Stemmed abstract'])

In [2]:
#add topic csv with 30 entries
data_t  = pd.read_csv('/Users/urmi/Documents/NLP/Assignment_1/topic_data_new.csv', encoding = "ISO-8859-1",converters={'Stop_word_stemmed_description': eval})
#usecols=['Stop word Tokenized description']

In [3]:
daat_t = list(data_t['Stop_word_stemmed_description'])

In [ ]:
#daat_t[12]

In [ ]:
class TaggedDocumentIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
        #print(doc_list)
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc, tags=[self.labels_list[idx]])
 

docLabels = list(data['Article_Id'])
data_stem = list(data['Stemmed_abstract'])
sentences = TaggedDocumentIterator(data_stem, docLabels)


In [ ]:
model = Doc2Vec(size=100, window=10, min_count=5, workers=11,alpha=0.025, iter=20)
model.build_vocab(sentences)
model.train(sentences,total_examples=model.corpus_count, epochs=model.iter)

In [4]:
# Save the model 
#model.save('model_similarity_1.doc2vec')
# Load the model
model_doc = Doc2Vec.load('model_similarity_1.doc2vec')


In [5]:
from operator import itemgetter

def get_similar(model_id,topic,n):
#one = ['58-year-old', 'african-american', 'woman', 'present', 'ER', 'episod', 'pressing/burn', 'anterior', 'chest', 'pain', 'began', 'two', 'day', 'earlier', 'first', 'time', 'life', 'pain', 'start', 'walk', 'radiat', 'back', 'accompani', 'nausea', 'diaphoresi', 'mild', 'dyspnea', 'increas', 'inspir', 'latest', 'episod', 'pain', 'end', 'half', 'hour', 'prior', 'arriv', 'known', 'hypertens', 'obes', 'deni', 'smoke', 'diabet', 'hypercholesterolemia', 'famili', 'histori', 'heart', 'diseas', 'current', 'take', 'medic', 'physic', 'examin', 'normal', 'ekg', 'show', 'nonspecif', 'chang']
    new_doc_vec = model_id.infer_vector(daat_t[topic], steps=50, alpha=0.25)
    similars = model_id.docvecs.most_similar([new_doc_vec], topn = n)
    #print(similars)

    sorted_tuples = sorted(similars, key=itemgetter(1),reverse = True)
    #print(sorted_tuples)
    return sorted_tuples

In [ ]:
#get_similar(model_doc,0,20)

In [7]:
 #save result in txt file
    with open('doc2vec_result.txt','w') as file:
        file.write('TOPIC_NO Q0 PMCID RANK SCORE RUN_NAME\n')
        for idxx,question in enumerate(daat_t):
            similar_all = get_similar(model_doc,idxx,1000)
            #print(idx,similar_all)
            for index, tuple in enumerate(similar_all):
                a_id = tuple[0]
                a_score = tuple[1]
               # print(index,tuple)
                file.write(f'{idxx+1}\t{0}\t{a_id}\t{index}\t{a_score}\t{"DocVec"}\n')
           
           

In [ ]:
#print(similar_all)